```python
# Recommendation System
├── Content-based Filtering
│   ├── Underthesea
│   │   ├── Word Tokenization
│   │   ├── Part-of-speech Tagging
│   │   └── Text Preprocessing
│   ├── Gensim
│   │   ├── TF-IDF Model
│   │   ├── LSI/LDA Models
│   │   └── Document Similarity
│   └── Cosine Similarity
│       ├── Vector Representation
│       ├── Similarity Calculation
│       └── Recommendation Generation
└── Collaborative Filtering
    ├── SurPRISE
    │   ├── Algorithm Implementation
    │   ├── Model Training
    │   └── Rating Prediction
    └── PySpark ALS
        ├── Data Preparation
        ├── Model Training
        └── Recommendation Generation
```
Cấu trúc này thể hiện:

Hệ thống gợi ý (Recommendation System) gồm 2 hướng tiếp cận chính:
1. Content-based Filtering:
   
   - Underthesea: xử lý ngôn ngữ tự nhiên tiếng Việt
   - Gensim: xử lý văn bản và tính toán độ tương đồng
   - Cosine Similarity: đo lường độ tương đồng
2. Collaborative Filtering:
   
   - SurPRISE: thư viện cho collaborative filtering
   - PySpark ALS: triển khai ALS (Alternating Least Squares) trên Spark

UI cho Recommendation System:
```python
    📱 Giao diện chính
    ├── 🏷️ "Dành cho bạn" (Collaborative Filtering)
    │   ├── "Dựa trên lịch sử mua hàng của bạn"
    │   └── "Người dùng tương tự bạn cũng thích"
    │
    └── 🎯 "Sản phẩm tương tự" (Content-based Filtering)
        ├── "Dựa trên sản phẩm bạn đang xem"
        └── "Phù hợp với sở thích của bạn"
```
```python
    📱 Trang chi tiết sản phẩm
    ├── Thông tin sản phẩm
    │
    ├── 🎯 "Sản phẩm tương tự" 
    │   └── (Dựa trên đặc điểm sản phẩm - Content-based)
    │
    └── 🏷️ "Có thể bạn cũng thích" 
        └── (Dựa trên hành vi người dùng - Collaborative)
```

```python
    📱 Trang chủ
    ├── 🏷️ "Top đề xuất cho bạn"
    │   └── (Kết hợp cả 2 phương pháp)
    │
    ├── 🎯 "Dựa trên sản phẩm bạn đã xem"
    │   └── (Content-based)
    │
    └── 🏷️ "Người dùng như bạn thường mua"
        └── (Collaborative)
```



### Recommendation System

#### Content-based Filtering


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
from gensim import corpora, models, similarities
import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
STOP_WORD_FILE = 'data/vietnamese-stopwords.txt'

def getStopWords():
    stop_words = []
    with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
        stop_words = file.read()

    stop_words = stop_words.split('\n')
    return stop_words


In [3]:
stop_words = getStopWords()
stop_words

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy',
 'ba',
 'ba_ba',
 'ba_bản',
 'ba_cùng',
 'ba_họ',
 'ba_ngày',
 'ba_ngôi',
 'ba_tăng',
 'bao_giờ',
 'bao_lâu',
 'bao_nhiêu',
 'bao_nả',
 'bay_biến',
 'biết',
 'biết_bao',
 'biết_bao_nhiêu',
 'biết_chắc',
 'biết_chừng_nào',
 'biết_mình',
 'biết_mấy',
 'biết_thế',
 'biết_trước',
 'biết_việc',
 'biết_đâu',
 'biết_đâu_chừng',
 'biết_đâu_đấy',
 'biết_được',
 'buổi',
 'buổi_làm',
 'buổi_mới',
 'buổi_ngày',
 'buổi_sớm',
 'bà',
 'bà_ấy',
 'bài',
 'bài_bác',
 'bài_bỏ',
 'bài_cái',
 'bác',
 'bán',
 'bán_cấp',
 'bán_dạ',
 'bán_thế',
 'bây_bẩy',
 'bây_chừ',
 'bây_giờ',
 'bây_nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên_bị',
 'bên_có',
 'bên_cạnh',
 'bông',
 'bước',
 'bước_khỏi',
 'bước_tới',
 'bước_đi',
 'bạn',
 'bản',
 'bản_bộ',
 'bản_riêng',
 'bản_thân',
 'bản_ý',
 'bất_chợt',
 'bất_cứ',
 'bất_giác',
 'bất_kì',
 'bất_kể',
 'bất_kỳ',
 'bất_luận',
 'bất_ngờ',
 'bất_nhược',
 'bất_quá',
 'bất_quá_chỉ',
 'bất_thình_l

In [4]:
df = pd.read_csv("data/Products_ThoiTrangNam_raw.csv")
df.head()

,product_id,product_name,category,sub_category,link,image,price,rating,description
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-ba-l%E1%BB%97-thun-g...,https://cf.shopee.vn/file/2c1ca03f5dc42f316fdf...,86250.0,4.9,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Tr...,https://cf.shopee.vn/file/c7ea4c6574dc79be6b26...,26800.0,4.9,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nXu...
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Ty...,https://cf.shopee.vn/file/6f93bcda10efe374f8cc...,39500.0,4.8,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nTh...
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81O-BA-L%E1%BB%96-H%C3%8...,https://cf.shopee.vn/file/1d7ed5e34bff8bc8b49a...,16500.0,4.8,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Thun-Nam-Th%E1%BB%83...,NaN,45000.0,4.8,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49663 entries, 0 to 49662
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    49663 non-null  int64  
 1   product_name  49663 non-null  object 
 2   category      49663 non-null  object 
 3   sub_category  49663 non-null  object 
 4   link          49663 non-null  object 
 5   image         36443 non-null  object 
 6   price         49663 non-null  float64
 7   rating        49663 non-null  float64
 8   description   48700 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 3.4+ MB


**Clean Data**

* Kiểm tra tính hợp lệ của URLs

In [6]:
import re

def is_valid_url(url):
    url_pattern = re.compile(
        r'^https?://'  # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+[A-Z]{2,6}\.?|'  # domain...
        r'localhost|'  # localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'  # ...or ip
        r'(?::\d+)?'  # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)
    return bool(url_pattern.match(url))

def validate_urls(df):
    # Find invalid URLs
    invalid_mask = ~df['link'].apply(is_valid_url)
    invalid_links = df[invalid_mask]
    
    print("\nInvalid URLs count:", len(invalid_links))
    
    if len(invalid_links) > 0:
        # Log invalid URLs for review
        print("Invalid URLs found:")
        for idx, row in invalid_links.iterrows():
            print(f"Product ID {row['product_id']}: {row['link']}")
        
        # Remove rows with invalid URLs
        df_cleaned = df[~invalid_mask].copy()
        return df_cleaned
    
    return df

# Usage
df = validate_urls(df)


Invalid URLs count: 0


* Kiểm tra tính hợp lệ của Price, Ratings

In [7]:
def validate_prices(df):
    negative_prices = (df['price'] < 0).sum()
    print(f"\nNegative prices: {negative_prices}")
    
    # Remove negative prices
    df_clean = df[df['price'] >= 0]
    
    return df_clean

def validate_ratings(df):
    invalid_ratings = ((df['rating'] < 0) | (df['rating'] > 5)).sum()
    print(f"\nInvalid ratings: {invalid_ratings}")
    
    print("\nRating statistics:")
    print(df['rating'].describe())
    
    print("\nRating distribution:")
    print(df['rating'].value_counts().sort_index())
    
    # Keep only valid ratings
    df_clean = df[(df['rating'] >= 0) & (df['rating'] <= 5)]
    
    return df_clean

# Usage
df = validate_prices(df)
df = validate_ratings(df)


Negative prices: 0

Invalid ratings: 0

Rating statistics:
count    49663.00000
mean         3.08549
std          2.33958
min          0.00000
25%          0.00000
50%          4.80000
75%          5.00000
max          5.00000
Name: rating, dtype: float64

Rating distribution:
0.0    17964
1.0      107
1.3        1
1.5        1
2.0       31
2.2        1
2.3        5
2.5        5
2.6        1
2.8        3
3.0      149
3.1        1
3.2        6
3.3       10
3.4        6
3.5       32
3.6       17
3.7       66
3.8       30
3.9       30
4.0      392
4.1       91
4.2      133
4.3      350
4.4      374
4.5      837
4.6     1217
4.7     2120
4.8     4282
4.9     6830
5.0    14571
Name: rating, dtype: int64


* Kiểm tra dữ liệu trùng và loại bỏ

In [8]:
def remove_duplicates(df):
    # Check for duplicates
    print("Total duplicate rows:", df.duplicated().sum())
    
    # Check duplicates based on specific columns
    print("\nDuplicates based on product_id:")
    print(df.duplicated(subset=['product_id']).sum())
    
    print("\nDuplicates based on product_name:")
    print(df.duplicated(subset=['product_name']).sum())
    
    # Remove duplicates based on product_id and product_name
    df_cleaned = df.drop_duplicates(subset=['product_id', 'product_name'])
    
    # Check the shape before and after removing duplicates
    print("\nOriginal dataset shape:", df.shape)
    print("Dataset shape after removing duplicates:", df_cleaned.shape)
    
    return df_cleaned

# Usage
df = remove_duplicates(df)

Total duplicate rows: 0

Duplicates based on product_id:
10

Duplicates based on product_name:
2560

Original dataset shape: (49663, 9)
Dataset shape after removing duplicates: (49653, 9)


* Kiểm tra từ có phải là từ tiếng Việt "sạch"

In [9]:
import re

# Hàm kiểm tra từ có phải là từ tiếng Việt "sạch"
def is_valid_vietnamese(word):
    vietnamese_chars = (
        "a-zA-Z0-9_"
        "àáạảãâầấậẩẫăằắặẳẵ"
        "èéẹẻẽêềếệểễ"
        "ìíịỉĩ"
        "òóọỏõôồốộổỗơờớợởỡ"
        "ùúụủũưừứựửữ"
        "ỳýỵỷỹ"
        "đ"
        "ÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴ"
        "ÈÉẸẺẼÊỀẾỆỂỄ"
        "ÌÍỊỈĨ"
        "ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ"
        "ÙÚỤỦŨƯỪỨỰỬỮ"
        "ỲÝỴỶỸ"
        "Đ"
    )
    pattern = f'^[{vietnamese_chars}]+$'
    return re.match(pattern, word) is not None

# Hàm xử lý một mô tả
def filter_vietnamese_words(text):
    if not isinstance(text, str):
        return ''
    words = text.split()
    clean_words = [w for w in words if is_valid_vietnamese(w)]
    return ' '.join(clean_words)


# Áp dụng lọc với progress bar
tqdm.pandas(desc="Filtering Vietnamese words")
df['description_clean'] = df['description'].progress_apply(filter_vietnamese_words)

df[['description', 'description_clean']]

Filtering Vietnamese words: 100%|██████████| 49653/49653 [00:07<00:00, 6472.04it/s]


,description,description_clean
0,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...,Danh Mục Shopee Thời Trang Nam Áo Ba Lỗ Chiều ...
1,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nXu...,Danh Mục Shopee Thời Trang Nam Áo Ba Lỗ Xuất x...
2,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nTh...,Danh Mục Shopee Thời Trang Nam Áo Ba Lỗ Thương...
3,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...,Danh Mục Shopee Thời Trang Nam Áo Ba Lỗ Chất l...
4,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...,Danh Mục Shopee Thời Trang Nam Áo Ba Lỗ Chiều ...
...,...,...
49658,Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nChiề...,Danh Mục Shopee Thời Trang Nam Chiều dài vớ Bắ...
49659,Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nThươ...,Danh Mục Shopee Thời Trang Nam Thương hiệu Liv...
49660,Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nChất...,Danh Mục Shopee Thời Trang Nam Chất liệu Cotto...
49661,Danh Mục\nShopee\nThời Trang Nam\nVớ/Tất\nKho ...,Danh Mục Shopee Thời Trang Nam Kho hàng 189 Gử...


* Lấy thông tin cần thiết để tạo thành cột `Content`

In [10]:
def combine_product_info(product_name, description, max_desc_words=100):
    # Lấy n từ đầu tiên từ mô tả
    truncated_desc = ' '.join(description.split()[:max_desc_words])
    # Kết hợp tên và mô tả
    return f"{product_name} {truncated_desc}"

# Áp dụng hàm cho DataFrame
df['Content'] = df.apply(
    lambda row: combine_product_info(
        row['product_name'], 
        row['description_clean'],
        100
    ), 
    axis=1
)
df['Content']

0        Áo ba lỗ thun gân ,form body tôn dáng Danh Mục...
1        Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu ...
2        Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...
3        ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON Danh Mục Sh...
4        Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...
                               ...                        
49658    Tất vớ nam cổ cao cao cấp, tất vớ đi giày công...
49659    Tất Nam Livan Sport Chất Liệu Bamboo Kháng Khu...
49660    Combo 3 Đôi Tất ( Vớ ) Hài Chống Hôi Chân Uni ...
49661    Hộp 10 đôi tất chống hôi chân nam Danh Mục Sho...
49662    Combo 5 đôi tất nam nữ cao cấp không hôi chân ...
Name: Content, Length: 49653, dtype: object

##### Underthesea

* Preprocess Vietnamese text bằng word_tokenize và loại bỏ stop_words để tạo thành cột `Content_wt`

In [11]:
def preprocess_vietnamese_text(text, stop_words):
    # Step 1: Word tokenization for Vietnamese text
    tokenized_text = word_tokenize(text, format="text")
    
    # Step 2: Remove Vietnamese stopwords and join words
    cleaned_text = ' '.join([word for word in tokenized_text.split() if not word in stop_words])
    
    return cleaned_text

# Apply the function with progress bar
tqdm.pandas(desc="Processing Vietnamese text")
df["Content_wt"] = df["Content"].progress_apply(lambda x: preprocess_vietnamese_text(x, stop_words))

# Display results
print("\nSample of processed texts:")
df[["Content", "Content_wt"]].head()

Processing Vietnamese text: 100%|██████████| 49653/49653 [03:10<00:00, 260.17it/s] 


Sample of processed texts:


,Content,Content_wt
0,"Áo ba lỗ thun gân ,form body tôn dáng Danh Mục...","Áo lỗ thun gân , form body tôn_dáng Danh_Mục S..."
1,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu ...","Áo Ba_Lỗ Nam_Trắng Chất_Cotton Siêu_Mát , Siêu..."
2,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...","Áo Ba_Lỗ Nam_Tyasuo chất vải co_dãn mát , khôn..."
3,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON Danh Mục Sh...,ÁO BA_LỖ HÀNG_VIỆT_NAM 100 % COTTON Danh_Mục S...
4,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Áo Thun_Nam Thể_Thao Ba_Lỗ Mẫu_Mới Siêu_Đẹp ( ...


* Preprocess text: remove number, remove special characters and convert to lowercase

In [12]:
# Tiền xử lý dữ liệu
def preprocess_text(content):
    # Remove numbers
    content_clean = [[re.sub('[0-9]+','', e) for e in text] for text in content]
    
    # Remove special characters and convert to lowercase
    special_chars = ['', ' ', ',', '.', '...', '-', ':', ';', '?', '%', '(', ')', '+', '/', "'", '&']
    content_clean = [[t.lower() for t in text if not t in special_chars] for text in content_clean]
    
    return content_clean

# Tokenize(split) the sentences into words
content = [[text for text in x.split()] for x in df.Content_wt]
content_re = preprocess_text(content)
content_re[:1]

[['áo',
  'lỗ',
  'thun',
  'gân',
  'form',
  'body',
  'tôn_dáng',
  'danh_mục',
  'shopee',
  'thời_trang',
  'nam_áo',
  'ba_lỗ',
  'chiều',
  'tay_áo',
  'khác',
  'phong_cách',
  'thể_cơ_đường',
  'nhiệt_đới',
  'tall_fit',
  'có',
  'xuất_xứ',
  'việt_nam_dáng',
  'kiểu',
  'áo',
  'ôm_sát',
  'chất_liệu',
  'cotton_mẫu',
  'sọc_tăm',
  'gân',
  'nổi',
  'kho',
  'hàng',
  'gửi',
  'bà_rịa',
  'vũng_tàu',
  'best',
  'tank_tops',
  'for_men',
  'áo_ba',
  'lỗ_quốc_dân',
  'cho_nam',
  'bảng_size',
  'size_m',
  'size_l',
  'kg',
  'size_xl',
  'kg',
  'size_xxl',
  'kg',
  'kg',
  'size',
  'chuẩn',
  'mặc',
  'kiểu',
  'body_nha',
  'quý_khách',
  'không',
  'thích',
  'mặc',
  'ôm']]

##### Gensim

* Tạo Dictionary, Corpus

In [21]:
def create_dictionary_and_corpus(content_re):
    # Create dictionary
    dictionary = corpora.Dictionary(content_re)
    feature_cnt = len(dictionary.token2id)
    
    # Create corpus
    corpus = [dictionary.doc2bow(text) for text in content_re]
    
    return dictionary, corpus, feature_cnt

# Usage
dictionary, corpus, feature_cnt = create_dictionary_and_corpus(content_re)

# Print dictionary and corpus information
print(f"Token to ID mappings:")
print(dictionary.token2id)
print(f"\nNumber of features (unique words): {feature_cnt}")
print(f"\nCorpus size (number of documents): {len(corpus)}")
print(f"Sample document vector (token_id, frequency): {corpus[0]}")

Token to ID mappings:
{'ba_lỗ': 0, 'best': 1, 'body': 2, 'body_nha': 3, 'bà_rịa': 4, 'bảng_size': 5, 'chiều': 6, 'cho_nam': 7, 'chuẩn': 8, 'chất_liệu': 9, 'cotton_mẫu': 10, 'có': 11, 'danh_mục': 12, 'for_men': 13, 'form': 14, 'gân': 15, 'gửi': 16, 'hàng': 17, 'kg': 18, 'kho': 19, 'khác': 20, 'không': 21, 'kiểu': 22, 'lỗ': 23, 'lỗ_quốc_dân': 24, 'mặc': 25, 'nam_áo': 26, 'nhiệt_đới': 27, 'nổi': 28, 'phong_cách': 29, 'quý_khách': 30, 'shopee': 31, 'size': 32, 'size_l': 33, 'size_m': 34, 'size_xl': 35, 'size_xxl': 36, 'sọc_tăm': 37, 'tall_fit': 38, 'tank_tops': 39, 'tay_áo': 40, 'thun': 41, 'thích': 42, 'thể_cơ_đường': 43, 'thời_trang': 44, 'tôn_dáng': 45, 'việt_nam_dáng': 46, 'vũng_tàu': 47, 'xuất_xứ': 48, 'áo': 49, 'áo_ba': 50, 'ôm': 51, 'ôm_sát': 52, 'azman_địa_chỉ': 53, 'chất_cotton': 54, 'co_giãn': 55, 'cotton': 56, 'giúp': 57, 'hà_nội': 58, 'hà_đông': 59, 'hôi': 60, 'hút': 61, 'khoắn': 62, 'khó_chịu': 63, 'khỏe': 64, 'lằn': 65, 'mái': 66, 'mềm_thoáng': 67, 'mổ': 68, 'nam_giới': 69, '

* Tạo hàm tính TF-IDF, Similarity Matrix

In [22]:
def create_tfidf_similarity_matrix(corpus, feature_cnt):
    # Create TF-IDF model
    tfidf = models.TfidfModel(corpus)
    
    # Create similarity matrix
    similarity_matrix = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features=feature_cnt)
    
    return tfidf, similarity_matrix

# Usage
tfidf, similarity_matrix = create_tfidf_similarity_matrix(corpus, feature_cnt)
print("Similarity matrix Length:", len(similarity_matrix))

Similarity matrix Length: 49653


* Tạo hàm `get_content_based_recommendations`

In [31]:
def get_similarity_scores(sims, top_k, exclude_idx=None):
    """
    Process similarity scores and return top K recommendations
    """
    sim_scores = list(enumerate(sims))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    if exclude_idx is not None:
        sim_scores = [s for s in sim_scores if s[0] != exclude_idx]
    
    return sim_scores[:top_k]

def format_recommendations(df, sim_scores):
    """
    Format recommendations into DataFrame
    """
    product_indices = [i[0] for i in sim_scores]
    recommendations = df.iloc[product_indices][['product_id', 'product_name', 'category', 'sub_category', 'price', 'rating']]
    recommendations['similarity_score'] = [i[1] for i in sim_scores]
    return recommendations

def process_keyword_query(query, dictionary, stop_words):
    """
    Process and validate keyword query
    """
    query = query.lower().strip()
    query_tokens = word_tokenize(query)
    
    query_tokens = [token for token in query_tokens 
                   if token not in stop_words 
                   and len(token) > 1]
    
    if len(query_tokens) == 0:
        raise ValueError("Query too short or contains only stop words")
    
    if len(query_tokens) > 10:
        print("Warning: Long queries may reduce accuracy. Consider using fewer terms.")
    
    query_bow = dictionary.doc2bow(query_tokens)
    if len(query_bow) == 0:
        raise ValueError("No valid keywords found after processing")
        
    return query_bow

def get_content_based_recommendations(query, index, tfidf, corpus, df, dictionary, top_k=5, by_keyword=False):
    """
    Get content-based recommendations for either a product ID or keyword search
    """
    if by_keyword:
        query_bow = process_keyword_query(query, dictionary, stop_words)
        sims = index[tfidf[query_bow]]
        sim_scores = get_similarity_scores(sims, top_k)
    else:
        product_idx = df[df['product_id'] == query].index[0]
        doc_vector = corpus[product_idx]
        sims = index[tfidf[doc_vector]]
        sim_scores = get_similarity_scores(sims, top_k, exclude_idx=product_idx)
    
    return format_recommendations(df, sim_scores)

* Lấy Recommendation cho product_id

In [32]:
# Example usage for product ID:
id_search = 19656

recommendations_by_id = get_content_based_recommendations(
    query=id_search,
    index=similarity_matrix,
    tfidf=tfidf,
    corpus=corpus,
    df=df,
    dictionary=dictionary,
    top_k=5,
    by_keyword=False
)
# For product ID based recommendations
print("Recommendations based on Product ID 19656:")
print("\nOriginal product:")
print(df[df['product_id'] == id_search][['product_name', 'category', 'sub_category']].iloc[0])
print("\nRecommended products:")
print(recommendations_by_id.to_string(index=False))

print("\n" + "="*80 + "\n")

Recommendations based on Product ID 19656:

Original product:
product_name    (A393) DỆT KIM ĐÔNG XUÂN ÁO MAY Ô NAM 3 LỖ MÀU...
category                                           Thời Trang Nam
sub_category                                             Áo Ba Lỗ
Name: 1156, dtype: object

Recommended products:
 product_id                                                                                                      product_name       category sub_category   price  rating  similarity_score
      19126                                                                       Áo Ba Lỗ Nam 100% Cotton Mặc nhà Thoáng Mát Thời Trang Nam     Áo Ba Lỗ 27000.0     4.9          0.229455
       1956                                                               [Xả Kho]Áo Ba Lỗ Nam 100% Cotton Mặc nhà Thoáng Mát Thời Trang Nam     Áo Ba Lỗ 33000.0     4.9          0.224185
      19304 Áo ba lỗ, áo ba lỗ nam cao cấp - Hàng Việt nam chất lượng cao 100% cotton, mặc mát vào mùa hè, ấm vào mùa đông .. T

* Lấy recommendation cho keyword


**Invalid Keywords Examples**:

```python
   keyword_search = "á"
   keyword_search = "và ai bà anh ba"
   # Result: ValueError: Query too short or contains only stop words
```

```python
   keyword_search = "áo ba lỗ nam cao cấp - Hàng Việt nam chất lượng cao 100% cotton, mặc mát vào mùa hè, ấm vào mùa đông"
   # Result: Warning: Long queries may reduce accuracy. Consider using fewer terms.
```

**Valid Keywords Examples**:
```python
   # Basic search
   keyword_search = "áo"

   # Optimal length
   keyword_search = "áo thun nam cotton"

   # Maximum recommended length
   keyword_search = "áo thun nam cotton giá rẻ chất lượng tốt nhất việt nam"
```


In [48]:

# Example usage for keyword search:
keyword_search = "áo thun nam cotton"

recommendations_by_keyword = get_content_based_recommendations(
    query=keyword_search,
    index=similarity_matrix,
    tfidf=tfidf,
    corpus=corpus,
    df=df,
    dictionary=dictionary,
    top_k=5,
    by_keyword=True
)

# For keyword based recommendations
print("Recommendations for keyword:", keyword_search)
print(recommendations_by_keyword.to_string(index=False))

Recommendations for keyword: áo thun nam cotton
 product_id                                                                                        product_name       category            sub_category    price  rating  similarity_score
      16514                                                                                      Áo nam dài tay Thời Trang Nam Trang Phục Truyền Thống 150000.0     5.0          0.471693
     161861                                                                                      Áo tay dài nam Thời Trang Nam Trang Phục Truyền Thống 149000.0     0.0          0.471693
     244310                                                                                 Bộ quần áo thun Nam Thời Trang Nam            Đồ Hóa Trang      0.0     0.0          0.468519
     241323                                                                   Sét bộ thun gồm 2 áo 1 quần 115k Thời Trang Nam            Đồ Hóa Trang 115000.0     0.0          0.400203
      19188 [SALE] 99k

##### Cosin Similarity

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vector hóa nội dung
vectorizer = TfidfVectorizer(analyzer='word', stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(df['Content_wt'])

# Tính toán độ tương đồng
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Hàm đề xuất khách sạn
# # với mỗi sản phẩm, lấy nums sản phẩm tương quan nhất
def get_recommendations(id, cosine_sim=cosine_sim, nums=5):
    idx = df.index[df['product_id'] == id][0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:nums+1]  # Lấy nums sp tương tự nhất
    product_indices = [i[0] for i in sim_scores]
    return df['product_name'].iloc[product_indices]

In [ ]:
# Gọi function
recommendations = get_recommendations(19656,cosine_sim,5)
print(recommendations)

1692    Áo hoodie nam nữ, áo nỉ siêu dày form rộng in ...
1729    Áo sweater - AZtino, áo nỉ in hình mặt người M...
1520    Áo sweater nam TINOWEAR, áo nỉ nam form rộng i...
1693    Áo sweater nam đẹp - áo nỉ bông trơn hình icon...
1830    Áo nỉ sweater nam nữ cổ tròn dài tay ấm áp ABA...
Name: product_name, dtype: object


#### Collaborative Filtering

##### SurPRISE

##### PySpark ALS